In [1]:
from indexify import IndexifyClient, ExtractionGraph

In [2]:
client = IndexifyClient()

In [3]:
extraction_graph_spec = """
name: 'pdflearner'
extraction_policies:
   - extractor: 'tensorlake/paddleocr_extractor'
     name: 'pdf_to_text'
"""

In [4]:
extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
# client.create_extraction_graph(extraction_graph)

In [5]:
content_id = client.upload_file("pdflearner", "/home/quamer23nasim38/faster-patient-onboarding-from-medical-history-using-indexify/data/6000267600123615133.pdf")

In [6]:
client.wait_for_extraction(content_id)

Waiting for extraction to complete for content id:  b9a151d10a300f4a
Extraction completed for content id:  b9a151d10a300f4a


In [7]:
extracted_content = client.get_extracted_content(ingested_content_id=content_id, graph_name="pdflearner", policy_name="pdf_to_text")

In [8]:
extracted_content

[{'id': 'a0129d4fa0dd8f9',
  'mime_type': 'text/plain',
  'content': b"21/8/2024\nPersonal Medical History Template.\nDescription of problem or complaints:.\nDHBSFG DIKDAY\nhead ache\nBirth Date\nfever\n body pain\n10/8/2014\nvomiting\nHave you previously been treated for this.\ncondition? If yes, by whom?.\nno\nAre your present problems due to an injury? if.\nyes, please select from the list..\n Personal Injury\nIf this is an accident has it been reported? If\nyes, to whom?.\nAuto Carrier\nAre you now or have you ever been disabled?.\nNo\n! Jotform\nCreate your own automated PDFs with Jotform PDF Editor- It's free\nFamily History\nHabits\nDiabetes\nHeart Disease\nCoffee, # of cups per day.\n0\nFather\n Mother\nSmoking, # of packs per day.\nKidney Disease\n0.5\nMother\nAlcohol, # of drinks per week.\n0\nQuestions about Job.\nJob description:\nsoftwere developer\nHow many hours do you sit per day?\n8\nHow many hours do you stand per day?.\n2\nPlease mark if you ever had any of the follo

In [9]:
extracted_content[0]

{'id': 'a0129d4fa0dd8f9',
 'mime_type': 'text/plain',
 'content': b"21/8/2024\nPersonal Medical History Template.\nDescription of problem or complaints:.\nDHBSFG DIKDAY\nhead ache\nBirth Date\nfever\n body pain\n10/8/2014\nvomiting\nHave you previously been treated for this.\ncondition? If yes, by whom?.\nno\nAre your present problems due to an injury? if.\nyes, please select from the list..\n Personal Injury\nIf this is an accident has it been reported? If\nyes, to whom?.\nAuto Carrier\nAre you now or have you ever been disabled?.\nNo\n! Jotform\nCreate your own automated PDFs with Jotform PDF Editor- It's free\nFamily History\nHabits\nDiabetes\nHeart Disease\nCoffee, # of cups per day.\n0\nFather\n Mother\nSmoking, # of packs per day.\nKidney Disease\n0.5\nMother\nAlcohol, # of drinks per week.\n0\nQuestions about Job.\nJob description:\nsoftwere developer\nHow many hours do you sit per day?\n8\nHow many hours do you stand per day?.\n2\nPlease mark if you ever had any of the followin

In [17]:
schema = {
    'properties': {
        'invoice_number': {'title': 'Invoice Number', 'type': 'string'},
        'date': {'title': 'Date', 'type': 'string'},
        'account_number': {'title': 'Account Number', 'type': 'string'},
        'owner': {'title': 'Owner', 'type': 'string'},
        'address': {'title': 'Address', 'type': 'string'},
        'last_month_balance': {'title': 'Last Month Balance', 'type': 'string'},
        'current_amount_due': {'title': 'Current Amount Due', 'type': 'string'},
        'registration_key': {'title': 'Registration Key', 'type': 'string'},
        'due_date': {'title': 'Due Date', 'type': 'string'}
    },
    'required': ['invoice_number', 'date', 'account_number', 'owner', 'address', 'last_month_balance', 'current_amount_due', 'registration_key', 'due_date'],
    'title': 'User',
    'type': 'object'
}

examples = str([
    {
        "type": "object",
        "properties": {
            "employer_name": {"type": "string", "title": "Employer Name"},
            "employee_name": {"type": "string", "title": "Employee Name"},
            "wages": {"type": "number", "title": "Wages"},
            "federal_tax_withheld": {"type": "number", "title": "Federal Tax Withheld"},
            "state_wages": {"type": "number", "title": "State Wages"},
            "state_tax": {"type": "number", "title": "State Tax"}
            },
        "required": ["employer_name", "employee_name", "wages", "federal_tax_withheld", "state_wages", "state_tax"]
    },
    {
        "type": "object",
        "properties": {
            "booking_reference": {"type": "string", "title": "Booking Reference"},
            "passenger_name": {"type": "string", "title": "Passenger Name"},
            "flight_number": {"type": "string", "title": "Flight Number"},
            "departure_airport": {"type": "string", "title": "Departure Airport"},
            "arrival_airport": {"type": "string", "title": "Arrival Airport"},
            "departure_time": {"type": "string", "title": "Departure Time"},
            "arrival_time": {"type": "string", "title": "Arrival Time"}
            },
        "required": ["booking_reference", "passenger_name", "flight_number","departure_airport", "arrival_airport", "departure_time", "arrival_time"]
    }
])



extraction_graph_spec = """
name: 'invoice-learner'
extraction_policies:
  - extractor: 'tensorlake/paddleocr_extractor'
    name: 'pdf-extraction'
  - extractor: 'schema_extractor'
    name: 'text_to_json'
    input_params:
      service: 'openai'
      example_text: {examples}
content_source: 'invoice-learner'
"""

In [18]:
from indexify import IndexifyClient, ExtractionGraph
client = IndexifyClient()

extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
client.create_extraction_graph(extraction_graph)
content_id = client.upload_file("invoice-learner", "/home/quamer23nasim38/faster-patient-onboarding-from-medical-history-using-indexify/data/6000267600123615133.pdf")
print(content_id)
client.wait_for_extraction(content_id)
extracted_content = client.get_extracted_content(content_id=content_id, graph_name="invoice-learner", policy_name="text_to_json")
print(extracted_content)

ApiException: status: Aborted, message: "unable to create extraction policies: Extractor with name schema_extractor not found", details: [], metadata: MetadataMap { headers: {"content-type": "application/grpc", "date": "Wed, 21 Aug 2024 12:24:51 GMT", "content-length": "0"} }